##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Classify structured data with feature columns

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/structured_data/feature_columns">
    <img src="https://www.tensorflow.org/images/tf_logo_32px.png" />
    View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/structured_data/feature_columns.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/structured_data/feature_columns.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/structured_data/feature_columns.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

Note: If you are starting a new project to classify structured data, we recommend you use [preprocessing layers](https://www.tensorflow.org/tutorials/structured_data/preprocessing_layers).

This tutorial demonstrates how to classify structured data (e.g. tabular data in a CSV). We will use [Keras](https://www.tensorflow.org/guide/keras) to define the model, and [feature columns](https://www.tensorflow.org/guide/feature_columns) as a bridge to map from columns in a CSV to features used to train the model. This tutorial contains complete code to:

* Load a CSV file using [Pandas](https://pandas.pydata.org/).
* Build an input pipeline to batch and shuffle the rows using [tf.data](https://www.tensorflow.org/guide/datasets).
* Map from columns in the CSV to features used to train the model using feature columns.
* Build, train, and evaluate a model using Keras.

## The Dataset

We will use a simplified version of the PetFinder [dataset](https://www.kaggle.com/c/petfinder-adoption-prediction). There are several thousand rows in the CSV. Each row describes a pet, and each column describes an attribute. We will use this information to predict the speed at which the pet will be adopted.

Following is a description of this dataset. Notice there are both numeric and categorical columns. There is a free text column which we will not use in this tutorial.

Column | Description| Feature Type | Data Type
------------|--------------------|----------------------|-----------------
Type | Type of animal (Dog, Cat) | Categorical | string
Age |  Age of the pet | Numerical | integer
Breed1 | Primary breed of the pet | Categorical | string
Color1 | Color 1 of pet | Categorical | string
Color2 | Color 2 of pet | Categorical | string
MaturitySize | Size at maturity | Categorical | string
FurLength | Fur length | Categorical | string
Vaccinated | Pet has been vaccinated | Categorical | string
Sterilized | Pet has been sterilized | Categorical | string
Health | Health Condition | Categorical | string
Fee | Adoption Fee | Numerical | integer
Description | Profile write-up for this pet | Text | string
PhotoAmt | Total uploaded photos for this pet | Numerical | integer
AdoptionSpeed | Speed of adoption | Classification | integer

## Import TensorFlow and other libraries

In [1]:
!pip install sklearn

In [2]:
import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

## Use Pandas to create a dataframe

[Pandas](https://pandas.pydata.org/) is a Python library with many helpful utilities for loading and working with structured data. We will use Pandas to download the dataset from a URL, and load it into a dataframe.

In [8]:
import pathlib

csv_file = 'flip.csv'
dataframe = pd.read_csv(csv_file)

In [9]:
dataframe.head()

,,,,,,,,,,"rial PVC Heel Height 0 inch Outer Material Canvas Color Multicolor""",No rating available,No rating available.1,DEEP FASHION,"{""product_specification""=>[{""key""=>""Ideal For"", ""value""=>""Women""}, {""key""=>""Occasion"", ""value""=>""Casual""}, {""key""=>""Sole Material"", ""value""=>""PVC""}, {""key""=>""Heel Height"", ""value""=>""0 inch""}, {""key""=>""Outer Material"", ""value""=>""Canvas""}, {""key""=>""Color"", ""value""=>""Multicolor""}]}"
e2f49036751c03e6108e0af6954d5567,2016-03-03 23:09:59 +0000,http://www.flipkart.com/bgs-alloy-kada/p/itmeggze9s78qqxv?pid=BBAEGGZEDHNZRKWM,BGS Alloy Kada,"[""Jewellery >> Bangles, Bracelets & Armlets >> Bracelets >> BGS Alloy Kada""]",BBAEGGZEDHNZRKWM,999.0,399.0,"[""http://img5a.flixcart.com/image/bangle-bracelet-armlet/k/w/m/1305977-free-size-bgs-1-1100x1100-imaeggxdwjy4pw2g.jpeg"", ""http://img6a.flixcart.com/image/bangle-bracelet-armlet/k/w/m/1305977-free-size-bgs-1-original-imaeggxdwjy4pw2g.jpeg"", ""http://img5a.flixcart.com/image/bangle-bracelet-armlet/k/w/m/1305977-free-size-bgs-1-original-imaeggxdmvgzhhhe.jpeg"", ""http://img5a.flixcart.com/image/bangle-bracelet-armlet/k/w/m/1305977-free-size-bgs-1-original-imaeggxdvdmgvcmw.jpeg""]",False,Key Features of BGS Alloy Kada Ideal For: Wome...,No rating available,No rating available,BGS,"{""product_specification""=>[{""key""=>""Pearl Type..."
660e91db93e04b564565942f32a22244,2016-03-03 23:09:59 +0000,http://www.flipkart.com/proganix-agave-nectar-plus-silica-curling-cream-defrizzant-hair-styler/p/itmee3pesjmvtzhu?pid=HSYEE3PERJWWMYHB,Proganix Agave Nectar Plus Silica Curling Cream Plus Defrizzant Hair Styler,"[""Beauty and Personal Care >> Hair Care >> Hair Styling >> Proganix Hair Styling >> Proganix Agave Nectar Plus Silica Curling Cream ...""]",HSYEE3PERJWWMYHB,1550.0,1550.0,"[""http://img5a.flixcart.com/image/hair-styling/y/h/b/proganix-170-agave-nectar-plus-silica-curling-cream-plus-1100x1100-imaee87wepbxazwy.jpeg"", ""http://img5a.flixcart.com/image/hair-styling/y/h/b/proganix-170-agave-nectar-plus-silica-curling-cream-plus-original-imaee87wepbxazwy.jpeg""]",False,Specifications of Proganix Agave Nectar Plus S...,No rating available,No rating available,Proganix,"{""product_specification""=>{""key""=>""Quantity"", ..."
49907d6f72c714ea52571212dbd59900,2016-03-03 23:09:59 +0000,http://www.flipkart.com/rainfun-car-cover-800/p/itme6fqugah6ha3w?pid=CCVE6FQUJHNHQVNQ,Rainfun Car Cover For 800,"[""Automotive >> Accessories & Spare parts >> Car & Bike Care >> Car & Bike Covers >> Car Body Covers >> Rainfun Car Cover For 800 (Without Mirror Pockets)""]",CCVE6FQUJHNHQVNQ,1799.0,599.0,"[""http://img6a.flixcart.com/image/car-cover/6/x/g/rf8067-rainfun-1100x1100-imae6feupfy7gnmz.jpeg"", ""http://img6a.flixcart.com/image/car-cover/6/x/g/rf8067-rainfun-original-imae6feupfy7gnmz.jpeg""]",False,Specifications of Rainfun Car Cover For 800 (W...,3.3,3.3,Rainfun,"{""product_specification""=>[{""key""=>""Brand"", ""v..."
d263f0803494b559817add0709a3090f,2016-03-03 23:09:59 +0000,http://www.flipkart.com/niremo-loafers/p/itmeg5xhzwecm6ag?pid=SHOEG5XH7GTM8D7Z,Niremo Loafers,"[""Footwear >> Women's Footwear >> Casual Shoes >> Loafers >> Niremo Loafers""]",SHOEG5XH7GTM8D7Z,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/q/t/h/peach-276-niremo-41-original-imaeg3p8ckpux47d.jpeg"", ""http://img6a.flixcart.com/image/shoe/q/t/h/peach-276-niremo-41-original-imaeg3p8buyf5zbw.jpeg"", ""http://img6a.flixcart.com/image/shoe/q/t/h/peach-276-niremo-41-original-imaeg3p8khfzthgb.jpeg"", ""http://img5a.flixcart.com/image/shoe/q/t/h/peach-276-niremo-41-original-imaeg3p8ykp3gezu.jpeg"", ""http://img6a.flixcart.com/image/shoe/q/t/h/peach-276-niremo-41-original-imaeg3p8jxxphf9z.jpeg""]",False,Specifications of Niremo Loafers General Occas...,No rating available,No rating available,Niremo,"{""product_specification""=>[{""key""=>""Occasion"",..."
c3a151b504c26a9ac7db271d98a0a32d,2016-03-03 23:09:59 +0000,http://www.flipkart.com/tab91-casual-short-sleeve

## Create target variable

The task in the original dataset is to predict the speed at which a pet will be adopted (e.g., in the first week, the first month, the first three months, and so on). Let's simplify this for our tutorial. Here, we will transform this into a binary classification problem, and simply predict whether the pet was adopted, or not.

After modifying the label column, 0 will indicate the pet was not adopted, and 1 will indicate it was.

In [13]:
# In the original dataset "4" indicates the pet was not adopted.
dataframe['target'] = np.where(dataframe['description'])


KeyError: ignored

## Split the dataframe into train, validation, and test

The dataset we downloaded was a single CSV file. We will split this into train, validation, and test sets.

In [10]:
train, test = train_test_split(dataframe, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

3698 train examples
925 validation examples
1156 test examples


## Create an input pipeline using tf.data

Next, we will wrap the dataframes with [tf.data](https://www.tensorflow.org/guide/datasets). This will enable us  to use feature columns as a bridge to map from the columns in the Pandas dataframe to features used to train the model. If we were working with a very large CSV file (so large that it does not fit into memory), we would use tf.data to read it from disk directly. That is not covered in this tutorial.

In [11]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [12]:
batch_size = 5 # A small batch sized is used for demonstration purposes
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

KeyError: ignored

## Understand the input pipeline

Now that we have created the input pipeline, let's call it to see the format of the data it returns. We have used a small batch size to keep the output readable.

In [ ]:
for feature_batch, label_batch in train_ds.take(1):
  print('Every feature:', list(feature_batch.keys()))
  print('A batch of ages:', feature_batch['Age'])
  print('A batch of targets:', label_batch )

We can see that the dataset returns a dictionary of column names (from the dataframe) that map to column values from rows in the dataframe.

## Demonstrate several types of feature columns
TensorFlow provides many types of feature columns. In this section, we will create several types of feature columns, and demonstrate how they transform a column from the dataframe.

In [14]:
# We will use this batch to demonstrate several types of feature columns
example_batch = next(iter(train_ds))[0]

NameError: ignored

In [ ]:
# A utility method to create a feature column
# and to transform a batch of data
def demo(feature_column):
  feature_layer = layers.DenseFeatures(feature_column)
  print(feature_layer(example_batch).numpy())

### Numeric columns
The output of a feature column becomes the input to the model (using the demo function defined above, we will be able to see exactly how each column from the dataframe is transformed). A [numeric column](https://www.tensorflow.org/api_docs/python/tf/feature_column/numeric_column) is the simplest type of column. It is used to represent real valued features. When using this column, your model will receive the column value from the dataframe unchanged.

In [ ]:
photo_count = feature_column.numeric_column('PhotoAmt')
demo(photo_count)

In the PetFinder dataset, most columns from the dataframe are categorical.

### Bucketized columns
Often, you don't want to feed a number directly into the model, but instead split its value into different categories based on numerical ranges. Consider raw data that represents a person's age. Instead of representing age as a numeric column, we could split the age into several buckets using a [bucketized column](https://www.tensorflow.org/api_docs/python/tf/feature_column/bucketized_column). Notice the one-hot values below describe which age range each row matches.

In [ ]:
age = feature_column.numeric_column('Age')
age_buckets = feature_column.bucketized_column(age, boundaries=[1, 3, 5])
demo(age_buckets)

### Categorical columns
In this dataset, Type is represented as a string (e.g. 'Dog', or 'Cat'). We cannot feed strings directly to a model. Instead, we must first map them to numeric values. The categorical vocabulary columns provide a way to represent strings as a one-hot vector (much like you have seen above with age buckets). The vocabulary can be passed as a list using [categorical_column_with_vocabulary_list](https://www.tensorflow.org/api_docs/python/tf/feature_column/categorical_column_with_vocabulary_list), or loaded from a file using [categorical_column_with_vocabulary_file](https://www.tensorflow.org/api_docs/python/tf/feature_column/categorical_column_with_vocabulary_file).

In [ ]:
animal_type = feature_column.categorical_column_with_vocabulary_list(
      'Type', ['Cat', 'Dog'])

animal_type_one_hot = feature_column.indicator_column(animal_type)
demo(animal_type_one_hot)

### Embedding columns
Suppose instead of having just a few possible strings, we have thousands (or more) values per category. For a number of reasons, as the number of categories grow large, it becomes infeasible to train a neural network using one-hot encodings. We can use an embedding column to overcome this limitation. Instead of representing the data as a one-hot vector of many dimensions, an [embedding column](https://www.tensorflow.org/api_docs/python/tf/feature_column/embedding_column) represents that data as a lower-dimensional, dense vector in which each cell can contain any number, not just 0 or 1. The size of the embedding (8, in the example below) is a parameter that must be tuned.

Key point: using an embedding column is best when a categorical column has many possible values. We are using one here for demonstration purposes, so you have a complete example you can modify for a different dataset in the future.

In [ ]:
# Notice the input to the embedding column is the categorical column
# we previously created
breed1 = feature_column.categorical_column_with_vocabulary_list(
      'Breed1', dataframe.Breed1.unique())
breed1_embedding = feature_column.embedding_column(breed1, dimension=8)
demo(breed1_embedding)

### Hashed feature columns

Another way to represent a categorical column with a large number of values is to use a [categorical_column_with_hash_bucket](https://www.tensorflow.org/api_docs/python/tf/feature_column/categorical_column_with_hash_bucket). This feature column calculates a hash value of the input, then selects one of the `hash_bucket_size` buckets to encode a string. When using this column, you do not need to provide the vocabulary, and you can choose to make the number of hash_buckets significantly smaller than the number of actual categories to save space.

Key point: An important downside of this technique is that there may be collisions in which different strings are mapped to the same bucket. In practice, this can work well for some datasets regardless.

In [ ]:
breed1_hashed = feature_column.categorical_column_with_hash_bucket(
      'Breed1', hash_bucket_size=10)
demo(feature_column.indicator_column(breed1_hashed))

### Crossed feature columns
Combining features into a single feature, better known as [feature crosses](https://developers.google.com/machine-learning/glossary/#feature_cross), enables a model to learn separate weights for each combination of features. Here, we will create a new feature that is the cross of Age and Type. Note that `crossed_column` does not build the full table of all possible combinations (which could be very large). Instead, it is backed by a `hashed_column`, so you can choose how large the table is.

In [ ]:
crossed_feature = feature_column.crossed_column([age_buckets, animal_type], hash_bucket_size=10)
demo(feature_column.indicator_column(crossed_feature))

## Choose which columns to use
We have seen how to use several types of feature columns. Now we will use them to train a model. The goal of this tutorial is to show you the complete code (e.g. mechanics) needed to work with feature columns. We have selected a few columns to train our model below arbitrarily.

Key point: If your aim is to build an accurate model, try a larger dataset of your own, and think carefully about which features are the most meaningful to include, and how they should be represented.

In [ ]:
feature_columns = []

# numeric cols
for header in ['PhotoAmt', 'Fee', 'Age']:
  feature_columns.append(feature_column.numeric_column(header))

In [ ]:
# bucketized cols
age = feature_column.numeric_column('Age')
age_buckets = feature_column.bucketized_column(age, boundaries=[1, 2, 3, 4, 5])
feature_columns.append(age_buckets)

In [ ]:
# indicator_columns
indicator_column_names = ['Type', 'Color1', 'Color2', 'Gender', 'MaturitySize',
                          'FurLength', 'Vaccinated', 'Sterilized', 'Health']
for col_name in indicator_column_names:
  categorical_column = feature_column.categorical_column_with_vocabulary_list(
      col_name, dataframe[col_name].unique())
  indicator_column = feature_column.indicator_column(categorical_column)
  feature_columns.append(indicator_column)

In [ ]:
# embedding columns
breed1 = feature_column.categorical_column_with_vocabulary_list(
      'Breed1', dataframe.Breed1.unique())
breed1_embedding = feature_column.embedding_column(breed1, dimension=8)
feature_columns.append(breed1_embedding)

In [ ]:
# crossed columns
age_type_feature = feature_column.crossed_column([age_buckets, animal_type], hash_bucket_size=100)
feature_columns.append(feature_column.indicator_column(age_type_feature))

### Create a feature layer
Now that we have defined our feature columns, we will use a [DenseFeatures](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/DenseFeatures) layer to input them to our Keras model.

In [ ]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

Earlier, we used a small batch size to demonstrate how feature columns worked. We create a new input pipeline with a larger batch size.

In [ ]:
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

## Create, compile, and train the model

In [ ]:
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dropout(.1),
  layers.Dense(1)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(train_ds,
          validation_data=val_ds,
          epochs=10)

In [ ]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

Key point: You will typically see best results with deep learning with much larger and more complex datasets. When working with a small dataset like this one, we recommend using a decision tree or random forest as a strong baseline. The goal of this tutorial is not to train an accurate model, but to demonstrate the mechanics of working with structured data, so you have code to use as a starting point when working with your own datasets in the future.

## Next steps
The best way to learn more about classifying structured data is to try it yourself. We suggest finding another dataset to work with, and training a model to classify it using code similar to the above. To improve accuracy, think carefully about which features to include in your model, and how they should be represented.